# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [2]:
html = requests.get(url).content

#miaumiau

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [4]:
from bs4 import BeautifulSoup

In [6]:
#los unique links que piden son mis ulinks
#hagamos la sopa:
soup = BeautifulSoup(html, "lxml")
ulinks = set(soup.findAll('a', href=True))
ulinks = list(set([link['href'] for link in ulinks]))
ulinks


['#cite_ref-:3_31-5',
 'https://www.stat.purdue.edu/~wsc/',
 '/wiki/William_S._Cleveland',
 'https://www.worldcat.org/oclc/489990740',
 '/w/index.php?title=Data_science&action=edit&section=9',
 '/wiki/Help:Contents',
 '#cite_note-27',
 '/wiki/Wikipedia:Contents',
 '/wiki/Q-learning',
 '/wiki/Dimensionality_reduction',
 '/wiki/Journal_of_Machine_Learning_Research',
 '/wiki/Mathematics',
 'http://simplystatistics.org/2013/12/12/the-key-word-in-data-science-is-not-data-it-is-science/',
 'https://commons.wikimedia.org/wiki/Category:Data_science',
 '#cite_ref-11',
 '/wiki/Anomaly_detection',
 '#cite_ref-TansleyTolle2009_4-0',
 '/wiki/Temporal_difference_learning',
 '/wiki/Category:Use_dmy_dates_from_December_2012',
 '/wiki/C.F._Jeff_Wu',
 '/w/index.php?title=Data_science&action=edit',
 'https://cran.r-project.org/doc/FAQ/R-FAQ.html#What-is-R_003f',
 '/wiki/Information_science',
 '/wiki/TensorFlow',
 '/wiki/Vasant_Dhar',
 '/wiki/Relevance_vector_machine',
 '/wiki/Category:CS1_maint:_others',

In [23]:
len(ulinks)

399

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
#para hacer el add del domain al link:
domain = 'http://wikipedia.org'

In [8]:
absolute = [link for link in links if (link.startswith(('http'))) & ('%' not in link)]
relative = [domain + link for link in links if (link.startswith(('/'))) & ('%' not in link)]
links  = list(set(absolute + relative))
links

['http://wikipedia.org/wiki/Deep_learning',
 'http://wikipedia.org/wiki/Factor_analysis',
 'https://sites.engineering.ucsb.edu/~shell/che210d/python.pdf',
 'https://books.google.com/books?id=oGs_AQAAIAAJ',
 'http://wikipedia.org/wiki/Template:Machine_learning_bar',
 'http://wikipedia.org/wiki/Wikipedia:File_Upload_Wizard',
 'http://www.datascienceassn.org/about-data-science',
 'http://wikipedia.org/wiki/David_Donoho',
 'http://wikipedia.org/wiki/Non-negative_matrix_factorization',
 'http://wikipedia.org/wiki/John_Tukey',
 'http://wikipedia.org/wiki/Linear_regression',
 'http://wikipedia.org/wiki/Linear_discriminant_analysis',
 'http://wikipedia.org/w/index.php?title=Data_science&action=history',
 'http://wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding',
 'https://www.stat.purdue.edu/~wsc/',
 'https://www.worldcat.org/oclc/489990740',
 'https://www.wikidata.org/wiki/Special:EntityPage/Q2374463',
 'http://wikipedia.org/wiki/Decision_tree_learning',
 'http://wikipedia.org/w

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [27]:
import os

In [34]:
folder = './wikipedia'

if not os.path.exists(folder):
    os.makedirs(folder)

os.chdir(folder)
os.getcwd()

#saqué output muchas veces y se anexó al final otra carpeta... no sé cómo quitarlas

'/Users/enrique/Documents/Ironhack/datamex0320/module-1/lab-parallelization/your-code/wikipedia/wikipedia/wikipedia/wikipedia/wikipedia/wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [17]:

from slugify import slugify
#para hacer legible el código de html y así

In [37]:
def index_page(link):
    try:
        page = requests.get(link).content
        nombreArchivo = slugify(link).lower() + '.html'
        with open(directory + '/' + nombreArchivo, 'wb') as f:
            f.write(page)
    except:
        pass
    

In [38]:
index_page('https://en.wikipedia.org')


In [ ]:
nomb

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [40]:
from tqdm import tqdm
from tqdm import trange

In [20]:
%%time
for link in links:
    index_page(link)

print("Completando indexado de página")

''' for i in trange(len(combined)):
    index_page(combined[i]) '''

Completando indexado de página
CPU times: user 24.8 s, sys: 1.72 s, total: 26.5 s
Wall time: 3min 5s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [42]:
import multiprocessing as mp

mp.cpu_count()

4

In [44]:
%%time
pool = mp.Pool()
resultado = pool.map(index_page, links)
pool.close()
pool.join()
print("Page indexing complete!")

Page indexing complete!
CPU times: user 54 ms, sys: 35.9 ms, total: 89.9 ms
Wall time: 47.3 s
